# Functions

In [1]:
dict_1 = {str(i): i for i in range(10_000)}
list_1 = [1, 2, 'a', 'B']*10_000
list_2 = [2, 3, 1]*10_000
list_3 = [[1, 2], [3, 4]]*10_000
 
def read_dict_v1(key, d):
    return d.get(key)

def write_dict_v1(key, value, d):
    d.update({key: value})
    
    return d

def read_dict_v2(key, d):
    if key in d.keys():
        return d[key]
    else:
        return None

def write_dict_v2(key, value, d):
    def new_items():
        yield from d.items()
        yield key, value
    
    return dict(new_items())

def modify_list_v1(l):
    new_l = []
    for el in l:
        if isinstance(el, int):
            new_l.append(pow(el, 2))
        elif isinstance(el, str):
            new_l.append(el.capitalize())
    return new_l

def modify_list_v2(l):
    sqr = map(lambda el: pow(el, 2) if isinstance(el, int) else el, l)
    sqr_cap = map(lambda el: el.capitalize() if isinstance(el, str) else el, sqr)
    
    return(list(sqr_cap))

def list_sort_v1(l):
    return sorted(l)

def list_sort_v2(l):
    lc = l.copy()
    lc.sort()
    return lc

def flatten_list_v1(nested):
    flat = []
    for sublist in nested:
        for el in sublist:
            flat.append(el)
    return flat

def flatten_list_v2(nested):
    gen = (el for sublist in nested for el in sublist)
    
    return list(gen)

# Testing functions

In [2]:
def test_read_dict(read_dict_func):
    di = {'a': 1}
    
    assert read_dict_func('a', di) == 1, 'Failed to read'
    assert read_dict_func('b', di) == None, 'Didnt return None'

def test_write_dict(write_dict_func):
    di = {}
    write_dict_func('a', 1, di)
    
    assert write_dict_func('a', 1, di) == {'a': 1}, 'Failed to write'

def test_modify_list(modify_list_func):
    list_1 = [1, 2, 'a', 'B']
    
    assert modify_list_func(list_1) == [1, 4, 'A', 'B']
    
def test_list_sort(list_sort_func):
    list_2 = [2, 3, 1]
    
    assert list_sort_func(list_2) == sorted(list_2)
    
def test_flatten_list(list_flatten_func):
    list_3 = [[1, 2], [3, 4]]
    
    assert list_flatten_func(list_3) == [1, 2, 3, 4]


test_read_dict(read_dict_v1)
test_read_dict(read_dict_v2)

test_write_dict(write_dict_v1)
test_write_dict(write_dict_v2)

test_modify_list(modify_list_v1)
test_modify_list(modify_list_v2)

test_list_sort(list_sort_v1)
test_list_sort(list_sort_v2)

test_flatten_list(flatten_list_v1)
test_flatten_list(flatten_list_v2)

# Time profiling

In [3]:
print('Read dict:')
%timeit read_dict_v1('a', dict_1)
%timeit read_dict_v2('a', dict_1)
print('Write dict:')
%timeit write_dict_v1('a', 1, dict_1)
%timeit write_dict_v2('a', 1, dict_1)

Read dict:
277 ns ± 2.74 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
349 ns ± 6.76 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
Write dict:
515 ns ± 17.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.4 ms ± 9.48 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Второй способ чтения дольше, скорее всего, из-за проверки условия.
Второй способ записи содержит создание нового словаря.


In [4]:
%timeit modify_list_v1(list_1)
%timeit modify_list_v2(list_1)

25.9 ms ± 538 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
30.8 ms ± 514 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Сопоставимые времена. Но, второй способ, предполагает два прохода по списку.

In [5]:
%timeit list_sort_v1(list_2)
%timeit list_sort_v2(list_2)

2.25 ms ± 67.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.24 ms ± 55.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Можно сказать, одинаково. Похоже, реализуют один и тот же алгоритм.

In [20]:
%timeit flatten_list_v1(list_3)
%timeit flatten_list_v2(list_3)

6.87 ms ± 97.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
5.04 ms ± 60.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


С генератором побыстрее.

# Memory profiling

In [ ]:
%load_ext memory_profiler

In [5]:
%%writefile funcs_to_be_profiled.py
from memory_profiler import profile


dict_1 = {str(i): i for i in range(10_000)}
list_1 = [1, 2, 'a', 'B']*10_000
list_2 = [2, 3, 1]*10_000
list_3 = [[1, 2], [3, 4]]*10_000

@profile(precision=6)   
def read_dict_v1(key, d):
    return d.get(key)

@profile(precision=6)
def write_dict_v1(key, value, d):
    d.update({key: value})

@profile(precision=6)
def read_dict_v2(key, d):
    if key in d.keys():
        return d[key]
    else:
        return None

@profile(precision=6)
def write_dict_v2(key, value, d):
    def new_items():
        yield from d.items()
        yield key, value
    
    d = dict(new_items()) # !!!

@profile(precision=6)
def modify_list_v1(l):
    new_l = []
    for el in l:
        if isinstance(el, int):
            new_l.append(pow(el, 2))
        elif isinstance(el, str):
            new_l.append(el.capitalize())
    return new_l

@profile(precision=6)
def modify_list_v2(l):
    sqr = map(lambda el: pow(el, 2) if isinstance(el, int) else el, l)
    sqr_cap = map(lambda el: el.capitalize() if isinstance(el, str) else el, sqr)
    
    return(list(sqr_cap))

@profile(precision=6)
def list_sort_v1(l):
    return sorted(l)

@profile(precision=6)
def list_sort_v2(l):
    lc = l.copy()
    lc.sort()
    return lc

@profile(precision=6)
def flatten_list_v1(nested):
    flat = []
    for sublist in nested:
        for el in sublist:
            flat.append(el)
    return flat

@profile(precision=6)
def flatten_list_v2(nested):
    gen = (el for sublist in nested for el in sublist)
    
    return list(gen)

if __name__ == '__main__':
    read_dict_v1('a', dict_1)
    write_dict_v1('a', 1, dict_1)
    read_dict_v2('a', dict_1)
    write_dict_v2('a', 1, dict_1)
    
    modify_list_v1(list_1)
    modify_list_v2(list_1)
    
    list_sort_v1(list_2)
    list_sort_v2(list_2)
    
    flatten_list_v1(list_3)
    flatten_list_v2(list_3)
    

Overwriting funcs_to_be_profiled.py


In [7]:
!python funcs_to_be_profiled.py

main
Filename: funcs_to_be_profiled.py

Line #    Mem usage    Increment   Line Contents
     9  33.324219 MiB  33.324219 MiB   @profile(precision=6)   
    10                             def read_dict_v1(key, d):
    11  33.324219 MiB   0.000000 MiB       return d.get(key)


Filename: funcs_to_be_profiled.py

Line #    Mem usage    Increment   Line Contents
    13  33.332031 MiB  33.332031 MiB   @profile(precision=6)
    14                             def write_dict_v1(key, value, d):
    15  33.332031 MiB   0.000000 MiB       d.update({key: value})


Filename: funcs_to_be_profiled.py

Line #    Mem usage    Increment   Line Contents
    17  33.332031 MiB  33.332031 MiB   @profile(precision=6)
    18                             def read_dict_v2(key, d):
    19  33.332031 MiB   0.000000 MiB       if key in d.keys():
    20  33.332031 MiB   0.000000 MiB           return d[key]
    21                                 else:
    22                                     return None


Filename: